# 기초상품 추천 코드 구현해보기
주어진 데이터를 활용하여 유저 아이디와 원하는 제품 유형을 입력하면   
해당 유저와 유사도가 높은 유저의 평점 높은 제품을 추천해주는 함수 만들기

* 유저데이터 파일명 : user_data
* 제품데이터 파일명 : product_data

In [1]:
import numpy as np

In [2]:
import pandas as pd

#유저데이터 불러오기
u_data = 'user_data.csv'

#제품데이터 불러오기
p_data = 'product_data.csv'

In [3]:
df1=pd.read_csv(u_data)
df2=pd.read_csv(p_data)

In [4]:
#product_id 기준으로 유저,제품 데이터 합쳐서 df에 저장하고 user_id를 기준으로 오름차순 정렬

df = pd.merge(df1, df2, on='product_id', how='inner')  
df.head()

,user_id,rating,product_id,skin,product_name,product_type,target_skin_type,ingredient,Unnamed: 5,Unnamed: 6
0,1,1,20,민감성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...,NaN,NaN
1,3,2,20,민감성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...,NaN,NaN
2,4,2,20,건성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...,NaN,NaN
3,6,2,20,지성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...,NaN,NaN
4,7,4,20,복합성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...,NaN,NaN


In [5]:
#각 유저가 제품에 남긴 평점들로 이루어진 데이터 프레임(df_users) 만들기 인덱스:user_id, 컬럼명:product_id
df_users=df.pivot(index='user_id', columns='product_id', values='rating')
df_users.fillna(0, inplace=True)

df_users.head()

product_id,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,0.0,5.0,0.0,2.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2,2.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

/Users/mandu/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
# pivot을 이용하여 데이터프레임 재구조화
df_users = df.pivot(index='user_id', columns='product_id', values='rating')
df_users.fillna(0, inplace=True)

# 코사인 유사도 계산
cos_matrix = cosine_similarity(df_users)

print(cos_matrix)

[[1.         0.37756255 0.11836501 ... 0.30848211 0.30876888 0.33248992]
 [0.37756255 1.         0.10316111 ... 0.22016375 0.30076748 0.11435019]
 [0.11836501 0.10316111 1.         ... 0.3901178  0.2384219  0.27498715]
 ...
 [0.30848211 0.22016375 0.3901178  ... 1.         0.09955444 0.40333349]
 [0.30876888 0.30076748 0.2384219  ... 0.09955444 1.         0.23491702]
 [0.33248992 0.11435019 0.27498715 ... 0.40333349 0.23491702 1.        ]]


In [10]:
#위에서 계산한 cos_matrix를 데이터프레임으로 만듭니다. 
df_users_cosine=pd.DataFrame(cos_matrix, index=df_users.index, columns=df_users.index)
df_users_cosine.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.377563,0.118365,0.195347,0.465273,0.350255,0.369234,0.382708,0.209271,0.388365,...,0.434633,0.327872,0.595550,0.253139,0.292668,0.306267,0.256333,0.308482,0.308769,0.332490
2,0.377563,1.000000,0.103161,0.106745,0.226767,0.327382,0.163269,0.323855,0.210206,0.190810,...,0.244048,0.212365,0.221078,0.246255,0.231953,0.218187,0.195578,0.220164,0.300767,0.114350
3,0.118365,0.103161,1.000000,0.328517,0.265719,0.423921,0.313412,0.399815,0.202431,0.194410,...,0.362167,0.376298,0.201629,0.211564,0.175430,0.439941,0.336052,0.390118,0.238422,0.274987
4,0.195347,0.106745,0.328517,1.000000,0.306010,0.280929,0.385025,0.287416,0.299863,0.477361,...,0.165149,0.281387,0.224007,0.263200,0.185619,0.296223,0.209036,0.376882,0.164470,0.357317
5,0.465273,0.226767,0.265719,0.306010,1.000000,0.209787,0.217964,0.221134,0.097609,0.239447,...,0.267159,0.319854,0.243056,0.183303,0.338358,0.361589,0.202548,0.051116,0.259230,0.302407


## 입력한 아이디의 유저와 비슷한 유저가 만족한 제품 추천하는 함수 만들기
1. df_users_cosine에서 입력한 아이디와 유사도 높은 5명을 sim_users로 지정
2. df에서 user_id가 sim_users의 인덱스와 일치하는 값들을 sim_user_df에 할당
3. sim_user_df에서 입력한 product_type과 일치하는 값들을 지정하고 rating을 기준으로 내림차순한 결과를 products에 할당
4. products에서 rating 4점 이상인 값들만 다시 products에 할당
5. products에서 product_name이 중복인 것은 첫번째 값만 남김
6. products의 product_name을 데이터 프레임 result로 만듦 (컬럼명은 나와 비슷한 사용자가 만족한(입력한 제품타입)제품)
7. 함수 마지막에 print(result) 출력

## 여긴 초기 스켈레톤 코드

In [ ]:
#반드시 skeleton 코드에 맞춰서 작성하지 않아도 됨. 단, 작성한 코드에 대해 어떤 알고리즘으로 결과가 나오게 했는지 설명할 수 있어야함. 

def user_based_recommend(user_id, product_type):
    sim_users = 


    sim_user_df =  
    
    products = 
    products = 
    
    products.
    
    result = 



In [19]:
df2

,product_id,product_name,product_type,target_skin_type,ingredient,Unnamed: 5,Unnamed: 6
0,1.0,ample_1,앰플,복합성,"정제수, 프로판다이올, 펜틸렌글라이콜, 글리세린, 1,2-헥산다이올, 판테놀, 참마...",NaN,NaN
1,2.0,ample_2,앰플,건성,"소나무잎추출물(792,555ppm), 부틸렌글라이콜, 다이프로필렌글라이콜, 글리세레...",NaN,NaN
2,3.0,lotion_1,로션,복합성,"정제수, 프로판다이올, 하이드로제네이티드폴리데센, 스쿠알란, 폴리글리세릴-3다이스테...",NaN,NaN
3,4.0,cream_1,크림,지성,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",NaN,NaN
4,5.0,cream_2,크림,민감성,정제수 다이카프릴릴에터 판테놀 글리세린 펜틸렌글라이콜 폴리글리세릴-6다이스테아레이트...,NaN,NaN
...,...,...,...,...,...,...,...
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df1

,user_id,rating,product_id,skin
0,1,1,20,민감성
1,1,5,5,민감성
2,1,2,3,민감성
3,1,4,34,민감성
4,1,2,18,민감성
...,...,...,...,...
1704,100,2,44,지성
1705,100,1,28,지성
1706,100,2,30,지성
1707,100,2,40,지성


## 작성하다 말았음

In [ ]:
#반드시 skeleton 코드에 맞춰서 작성하지 않아도 됨. 단, 작성한 코드에 대해 어떤 알고리즘으로 결과가 나오게 했는지 설명할 수 있어야함. 

def user_based_recommend(user_id, product_type):
    sim_users = df_users_cosine[user_id]

    sim_user_df =  list(enumerate(product_type[idx]))
    
    products = 
    products = 
    
    products.
    
    result = 



## 이 코드가 예전에 영화추천 코드

In [ ]:

def similar_movie_1(title, cos_matrix):
    idx = indices[title]                #제목을 입력하면 해당 인덱스 값(몇번째 행인지)을 가져옴
    
    cos_sim = list(enumerate(cos_matrix[idx]))        #몇번째 행인지와 유사도를 같이 가져옴
#     return cos_sim
    cos_sim.sort(key=lambda x :x[1], reverse=True)   #유사도 값을 기준으로 내림차순 정렬
    
    cos_sim = cos_sim[0:11]                          #상위 11개 작품 (다음 코드에서 입력한 작품 제외할 것)   
    sim_movie_idx=[x[0] for x in cos_sim]            #cos_sim에서 첫번쨰 항목만 즉, 몇행인지 값 추출
    
    if idx in sim_movie_idx:                         #해당 작품 제외
        sim_movie_idx.remove(idx)
              
    sim_movie_idx = sim_movie_idx[0:10]              #상위 10개 작품만
#     return sim_movie_idx
    
    title_list=movie['title'].iloc[sim_movie_idx]  #인덱스로 유사한 영화 제목 추출
    title_list=pd.DataFrame(title_list)            #데이터프레임 만듦
#     return title_list
    
    id_list=movie['id'].iloc[sim_movie_idx]        #인덱스로 영화 id 추출
    id_list=pd.DataFrame(id_list)                  #데이터 프레임으로 만듦
#     return id_list

    temp1 = pd.concat([title_list,id_list], axis=1)                              #title_list, id_list 데이터프레임 병합
#     return temp1
    temp2=pd.merge(temp1, pred_rating["pred"],left_on = 'id',right_index=True)   #temp1과 pred_rating 병합

    result=temp2.drop(["id"], axis=1)
    result.rename(columns={"title":"유사한 영화 TOP10", "pred":"예상평점"},inplace=True)
    
    print(result)

    
   

<img src="./추천결과예시.PNG" width ="300px" height ="200px"></img>